# Spam Ham Detection Using BERT and Tensorflow

### <u>Project Summary</u>

### <u>GitHub Link</u>
[Click Here](https://github.com/ajitmane36/spam-ham-detection-bert-tensorflow.git)

### <u>Problem Statement</u>

- The data is related to the classification of emails into spam or ham (non-spam). The goal of this project is to develop a model using BERT and TensorFlow to predict whether an email is spam or not based on its content. By fine-tuning a pre-trained BERT model, the objective is to enhance the accuracy and efficiency of email classification, ensuring that legitimate emails are delivered to the inbox while spam is effectively filtered out.

### <u>Data Description</u>

- **text**: Description of the email content.
- **spam**: Indicates whether the email is spam (1) or not (0).